<a href="https://colab.research.google.com/github/DheniMoura/Uninter_Trabalho-Big-Data/blob/main/Atividade_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dhenifer Moura
ru = 452947

In [2]:
# importar o arquivo do google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Instalações necessárias
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:
#copia do dataframe na pasta drive
!cp /content/drive/MyDrive/Colab_Notebooks/imdb-reviews-pt-br.csv /content

cp: cannot stat '/content/drive/MyDrive/Colab_Notebooks/imdb-reviews-pt-br.csv': No such file or directory


In [6]:
#biblioteca "os" para configuração das variáveis de caminho de ambiente
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [7]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

In [8]:
# para manipular vairáveis
from pyspark.sql.functions import *

In [9]:
#sessão local
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Prática 01 – Somatório de IDs
Questão – Qual o valor da soma de todos os campos “id” dos filmes classificados como negativos para o banco de dados “imdb-reviews-pt-br.csv”<br>
Descobrir qual o valor da soma de todos os campos “id” dos filmes classificados como negativos para o banco de dados “imdb-reviews-pt-br.csv”.

In [10]:
# carregando dados
df = spark.read.csv("/content/drive/MyDrive/Uninter/Big_Data/trabalho_452947/imdb-reviews-pt-br.csv", 
                    header=True,quote="\"", escape="\"", encoding="UTF-8")

In [11]:
# visualização 
df.show(5)

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
+---+--------------------+--------------------+---------+
only showing top 5 rows



In [12]:
#tipo do dataframe
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
# id de string para inteiro
df = df.withColumn(
    colName = 'id', 
    col = regexp_replace('id','','').cast('int')
    )

df.select('id').printSchema()

root
 |-- id: integer (nullable = true)



In [15]:
#Filtrando por sentiment = neg
df_neg = df.where('sentiment == "neg" ')
df_neg.show(5)

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
+---+--------------------+--------------------+---------+
only showing top 5 rows



In [16]:
#somando a coluna id com base no sentiment neg.
print("RU: ", ru)
total_neg = df_neg.groupBy('sentiment').agg(sum('id')
.alias("Total_sum")).orderBy('sentiment').where('sentiment == "neg" ')
total_neg.show()

RU:  452947
+---------+---------+
|sentiment|Total_sum|
+---------+---------+
|      neg|459568555|
+---------+---------+



###Conclusão
Para os filmes classificados com ‘sentiment’ = ‘neg’, os valores dos ‘ids’ somaram 459.568.555

# Prática 02 – Diferença do número de palavras totais de português para inglês dos textos negativos
Questão – Contar palavras dos textos negativos e achar diferença de quantidade. <br>
Nessa prática você deverá contar todas as palavras existentes nos textos negativos (Português e Inglês) e então deverá encontrar quantas palavras a mais, no total, os textos em português possuem.
Para tal, crie um script em Python ou Scala e rode-o com sua máquina virtual Hadoop ou Spark ou PySpark, como feito na prática 1.
É necessário se preocupar em filtrar corretamente as avaliações de filmes para que apenas os textos marcados como negativos sejam contabilizados.

In [17]:
# criando dfs apenas com as colunas desejadas
df_en = df.select('id','text_en','sentiment')
df_pt = df.select('id','text_pt','sentiment')

In [18]:
# filtrando conde o sentiment é negativo
df_en = df_en.where('sentiment == "neg" ')
df_pt = df_pt.where('sentiment == "neg" ')

In [19]:
# deixando apenas os textos nos arquivos
df_en = df_en.drop('sentiment')
df_pt = df_pt.drop('sentiment')

In [20]:
#dividindo o texto em palavras
palavras_en = df_en.select("id", explode(split("text_en", "[ ]+")).alias("palavra"))
palavras_pt = df_pt.select("id", explode(split("text_pt", "[ ]+")).alias("palavra"))

In [21]:
# contando quantas vezes cada palavra aparece
soma_palavras_en = palavras_en.groupBy("palavra").count()
soma_palavras_pt = palavras_pt.groupBy("palavra").count()
print("RU: ", ru)
soma_palavras_en.show(5)
soma_palavras_pt.show(5)

RU:  452947
+-------+-----+
|palavra|count|
+-------+-----+
|  Aside|  117|
|    few| 3962|
|  still| 3684|
|  those| 3814|
|    ...|  597|
+-------+-----+
only showing top 5 rows

+---------+-----+
|  palavra|count|
+---------+-----+
|Kutchers.|    2|
|    podia|  469|
|      ...|12425|
|   exceto| 1193|
| chamavam|   16|
+---------+-----+
only showing top 5 rows



In [22]:
total_en = soma_palavras_en.agg(sum('count').alias("Total_palavras_en")).first()[0]
total_pt = soma_palavras_pt.agg(sum('count').alias("Total_palavras_pt")).first()[0]
print("RU: ", ru)
print(f'total_inglês: {total_en}\ntotal_português: {total_pt}')

RU:  452947
total_inglês: 5400276
total_português: 5455273


In [23]:
print("RU: ", ru)
print(f'Total de palavras em Português: {total_pt}\nTotal de palavras em Inglês: {total_en}')
print(f'Há {total_pt - total_en} mais palavras em português do que em inglês nesses textos.')

RU:  452947
Total de palavras em Português: 5455273
Total de palavras em Inglês: 5400276
Há 54997 mais palavras em português do que em inglês nesses textos.


###Conclusão
Existem 54.997 palavras a mais nos textos em português do que em inglês